## Importing Libraries and Preparing for Image Processing

In this part of the code, we begin by importing the necessary libraries:

- **`torch`**: This is the core PyTorch library, which is used for working with neural networks and tensor operations.
- **`torchvision.transforms`**: This module provides several image transformations that are applied to the image before feeding it into the model.
- **`PIL` (Python Imaging Library)**: This library is used for opening and manipulating image files.
- **`faiss`**: A library for efficient similarity search and clustering of dense vectors, used here for creating and searching image embeddings.
- **`pickle`**: This is used to serialize and save/load Python objects (e.g., embeddings) to/from disk.

### Image Transformations

We define a sequence of transformations that will be applied to images before feeding them into a neural network model for feature extraction. The transformation steps include:
- **Resizing**: The images are resized to 256x256 pixels to ensure consistent input size.
- **Center Cropping**: The center of the image is cropped to 224x224 pixels, a typical input size for models like ResNet.
- **ToTensor**: This converts the image into a PyTorch tensor, which is required for input into the neural network.
- **Normalization**: The image is normalized to have a mean and standard deviation that matches the pre-trained ResNet model’s expectations. This helps the model make predictions effectively since it was trained with these statistics.

### Loading the Pre-Trained Model

Here, we load a **ResNet-50** model pre-trained on the ImageNet dataset using `torch.hub.load`. This model is a powerful convolutional neural network (CNN) used for image classification tasks. We use the model in **evaluation mode** (`model.eval()`) to ensure that the model's behavior is optimized for inference, rather than training (which would involve gradients and backpropagation). 

This pre-trained model will be used to extract feature vectors (embeddings) from the images, which will then be used for matching similar artworks based on the image input provided by the user.

In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import faiss
import pickle

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load a pre-trained model (e.g., ResNet-50)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

Using cache found in /home/hamza-ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
/home/hamza-ubuntu/.conda/envs/image_similarity/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hamza-ubuntu/.conda/envs/image_similarity/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Extracting Image Features

The `extract_features` function extracts the feature vector (embedding) from a given image using a pre-trained ResNet-50 model.

### Function Breakdown:
1. **Input**: Takes the path to an image (`image_path`).
2. **Processing**: 
   - Opens the image and applies transformations like resizing, cropping, converting to a tensor, and normalizing it.
   - Checks if the image has 3 channels (RGB). If not, it skips the image.
3. **Feature Extraction**: Passes the image through the ResNet-50 model to get a feature vector, which is returned as a NumPy array.
4. **Error Handling**: If an error occurs (e.g., invalid image), it prints an error message and returns `None`.

This function is used to obtain feature embeddings, which are then used for image similarity matching.

In [2]:
def extract_features(image_path):
    """
    Extracts features (embeddings) from an image.

    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Extracted features as a numpy array, 
                    or None if there's an error.
    """
    try:
        img = Image.open(image_path)
        img_tensor = transform(img).unsqueeze(0)
        # Check if the tensor has 3 channels
        if img_tensor.shape[1] != 3:
            print(f"Image {image_path} has {img_tensor.shape[1]} channels, skipping.")
            return None

        with torch.no_grad():
            features = model(img_tensor).squeeze().numpy()
        return features
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

## Searching for Similar Images  

The `search_similar_images` function allows us to find the most similar images in the dataset based on a query image. It leverages the Faiss index, which we previously populated with image embeddings, to perform efficient similarity searches.  

### How It Works  

1. **Query Image Embedding**:  
   The function starts by extracting the feature embedding of the query image using the `extract_features` function. This transforms the image into a numerical representation that can be compared against other images in the dataset.  

2. **Performing the Search**:  
   The query embedding is reshaped and passed to the Faiss index using the `index.search()` method. This method retrieves the top `k` (in this case, 5) most similar images based on their Euclidean distance from the query image. The `distances` array holds the similarity scores, while `indices` contains the indices of the most similar images.  

3. **Mapping Indices to Image Paths**:  
   The function then maps the indices returned by Faiss back to the corresponding image file paths from the `image_paths` list. This gives us the file paths to the most similar images.  

### Output  
The function returns a list of file paths to the most similar images. This allows us to efficiently retrieve and display the images that are most closely related to the query, enabling an effective image search experience.

In [3]:
def search_similar_images(query_image_path, index, image_paths):
    """
    Searches for similar images based on a query image.

    Args:
        query_image_path (str): Path to the query image.
        index (faiss.Index): The Faiss index containing image embeddings.
        image_paths (list): List of image paths.

    Returns:
        list: A list of paths to the most similar images.
    """
    query_embedding = extract_features(query_image_path)
    distances, indices = index.search(query_embedding.reshape(1, -1), k=5)
    similar_image_paths = [image_paths[i] for i in indices[0]]
    return similar_image_paths

## Loading Embeddings  

- **`load_embeddings`**: This function retrieves the embeddings from a previously saved file. By opening the file in binary read mode (`rb`) and using `pickle.load`, it reconstructs the original embeddings for use in the image retrieval pipeline.  


In [ ]:
def load_embeddings(filenamindexe):
  """
  Loads embeddings from a pickle file.

  Args:
      filename (str): The filename to load the embeddings from.

  Returns:
      np.ndarray: The loaded embeddings.
  """
  with open(filename, 'rb') as f:
    return pickle.load(f)

## Running the Similarity Search

In this portion of the code, the program performs the image similarity search by loading the precomputed embeddings and Faiss index, then finding the most similar images to a query.

### Step-by-step Breakdown:
1. **Loading the Faiss Index**:
   - An index is created using `faiss.IndexFlatL2` to store the image embeddings based on the feature size output by the ResNet-50 model.
   - The precomputed embeddings (`mega_embeddings.pkl`) are loaded using `load_embeddings` and added to the Faiss index with the `index.add()` method. This step makes the embeddings searchable for future queries.

2. **Loading Image Paths**:
   - The image paths are read from a CSV file (`image_paths.csv`) that contains the list of all images in the dataset. These paths are loaded into the `image_paths_loaded` list, which will be used to map the indices of the similar images found by the search.

3. **Query Image**:
   - A query image (`vase.jpeg`) is specified. The `search_similar_images` function is called to find the most similar images to the query image based on the embeddings stored in the Faiss index.

4. **Searching and Displaying Results**:
   - The `search_similar_images` function returns a list of the most similar image paths, which are then printed out for the user.

This process enables the system to perform efficient image similarity searches, identifying the artworks that most closely resemble a given query image.

In [ ]:
import csv
index = faiss.IndexFlatL2(model.fc.out_features)
embeddings = load_embeddings('artifacts_image_retrieval/mega_embeddings.pkl')
index.add(embeddings)
with open('artifacts_image_retrieval/image_paths.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    image_paths_loaded = [row[0] for row in reader]
query_image_path = "test_image/vase.jpeg"
similar_images = search_similar_images(query_image_path, index, image_paths_loaded)
print(similar_images)

['SemArt/Images/08686-10vase.jpg', 'SemArt/Images/34434-stillif.jpg', 'SemArt/Images/44771-still_li.jpg', 'SemArt/Images/31879-y_woman1.jpg', 'SemArt/Images/34462-18rippl.jpg']
